In [2]:
import pandas as pd

In [3]:
# the baseball.csv file (included in the project directory) is read into the dataframe
url = 'https://drive.google.com/uc?id=1qquqUziZdQGYLXBTaQ6EaGVGbpbiWWzs'
baseball = pd.read_csv(url)

In [4]:
baseball.head()

,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG
0,ARI,NL,2012,734,688,81,0.328,0.418,0.259,0,NaN,NaN,162,0.317,0.415
1,ATL,NL,2012,700,600,94,0.320,0.389,0.247,1,4.0,5.0,162,0.306,0.378
2,BAL,AL,2012,712,705,93,0.311,0.417,0.247,1,5.0,4.0,162,0.315,0.403
3,BOS,AL,2012,734,806,69,0.315,0.415,0.260,0,NaN,NaN,162,0.331,0.428
4,CHC,NL,2012,613,759,61,0.302,0.378,0.240,0,NaN,NaN,162,0.335,0.424


In [5]:
baseball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Team          1232 non-null   object 
 1   League        1232 non-null   object 
 2   Year          1232 non-null   int64  
 3   RS            1232 non-null   int64  
 4   RA            1232 non-null   int64  
 5   W             1232 non-null   int64  
 6   OBP           1232 non-null   float64
 7   SLG           1232 non-null   float64
 8   BA            1232 non-null   float64
 9   Playoffs      1232 non-null   int64  
 10  RankSeason    244 non-null    float64
 11  RankPlayoffs  244 non-null    float64
 12  G             1232 non-null   int64  
 13  OOBP          420 non-null    float64
 14  OSLG          420 non-null    float64
dtypes: float64(7), int64(6), object(2)
memory usage: 144.5+ KB


As can be seen from the `.info()` method output, while the dataframe as read from the .csv file is already quite well organized, some improvements can be made. Moreover, columns `RankSeason`, `RankPlayoffs`, `OOBP`, `OSLG` seem to be missing data.

## Checking for missing data


The data encompasses years 1962-2012, which, at a glance, amounts to 51 seasons of baseball.
Based on the state of the MLB today, the above numbers would imply that some data is missing for all columns (since 30 teams x 51 years = 1530, not 1232). Additionally, 4 columns seem to be missing even more data: `RankSeason` and `RankPlayoffs` have only 244 entries each, while `OOBP` and `OSLG` - 420 per column.

However, here are some things to consider that explain the discrepancies:
* Major League Baseball was not always made up of 30 teams. Indeed, in the first year in this dataset (1962), the league consisted of only 20 teams:

In [6]:
len(baseball.Year[baseball.Year==1962])

20

We can easily find the number of teams for each year in the dataset and compare with (presumably accurate) information from [Baseball Reference](https://www.baseball-reference.com/leagues/majors/bat.shtml), according to which, there were 20 teams in years '62-'68; 24 from '69-'76; 26: '77-'92; 28: '93-'98, and 30 from '98 onwards. The following code confirms that the dataset matches reality in this respect: 

In [7]:
for x in range(1962, 2013):
  print(f'{x}: {len(baseball.Year[baseball.Year==x])}')

1962: 20
1963: 20
1964: 20
1965: 20
1966: 20
1967: 20
1968: 20
1969: 24
1970: 24
1971: 24
1972: 0
1973: 24
1974: 24
1975: 24
1976: 24
1977: 26
1978: 26
1979: 26
1980: 26
1981: 0
1982: 26
1983: 26
1984: 26
1985: 26
1986: 26
1987: 26
1988: 26
1989: 26
1990: 26
1991: 26
1992: 26
1993: 28
1994: 0
1995: 0
1996: 28
1997: 28
1998: 30
1999: 30
2000: 30
2001: 30
2002: 30
2003: 30
2004: 30
2005: 30
2006: 30
2007: 30
2008: 30
2009: 30
2010: 30
2011: 30
2012: 30


* The zeroes in the above list indicate another chunk of missing rows: the years 1972, 1981, 1994, and 1995 had a strike which caused the season to be incomplete, making its data incomplete for the purpose of this dataset.

* `RankSeason` and `RankPlayoffs` don't have entries for every row - these columns are significantly shorter because they only contain data relevant to playoff teams.

* `OOBP` and `OSLG` are, respectively, "Opponents' On-Base Percentage" and "Opponents' Slugging Percentage", which only seem to have been tracked since 1999. For the seasons that this data is present, however, it is complete: 14 seasons between '99-'12 x 30 teams = 420.



# Re-typing columns appropriately

Pandas is able to recognize the values in the .csv file appropriately for the most part. However, there are still a few things that could be changed.

For one, `Team` and `League` columns are categorical data (drawing from a pool of 39 and 2 possibilities, respectively, as below) and thus might be better served as `category` type instead of `object`.

In [8]:
set(baseball.League)

{'AL', 'NL'}

In [9]:
set(baseball.Team)

{'ANA',
 'ARI',
 'ATL',
 'BAL',
 'BOS',
 'CAL',
 'CHC',
 'CHW',
 'CIN',
 'CLE',
 'COL',
 'DET',
 'FLA',
 'HOU',
 'KCA',
 'KCR',
 'LAA',
 'LAD',
 'MIA',
 'MIL',
 'MIN',
 'MLN',
 'MON',
 'NYM',
 'NYY',
 'OAK',
 'PHI',
 'PIT',
 'SDP',
 'SEA',
 'SEP',
 'SFG',
 'STL',
 'TBD',
 'TBR',
 'TEX',
 'TOR',
 'WSA',
 'WSN'}

In [10]:
len(set(baseball.Team))

39

In [11]:
baseball['Team'] = baseball.Team.astype('category')
baseball['League'] = baseball.League.astype('category')

The `Playoffs` column, which takes a value 1 if a team made the playoffs at the end of the season and 0 otherwise, can also be replaced with a boolean type(this is based on the assumption that boolean values will not create issues later on at the analysis step):

In [12]:
baseball.Playoffs = [x == 1 for x in baseball.Playoffs]

Or, more simply, with:

In [13]:
baseball['Playoffs'] = baseball.Playoffs.astype(bool)

In [14]:
baseball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Team          1232 non-null   category
 1   League        1232 non-null   category
 2   Year          1232 non-null   int64   
 3   RS            1232 non-null   int64   
 4   RA            1232 non-null   int64   
 5   W             1232 non-null   int64   
 6   OBP           1232 non-null   float64 
 7   SLG           1232 non-null   float64 
 8   BA            1232 non-null   float64 
 9   Playoffs      1232 non-null   bool    
 10  RankSeason    244 non-null    float64 
 11  RankPlayoffs  244 non-null    float64 
 12  G             1232 non-null   int64   
 13  OOBP          420 non-null    float64 
 14  OSLG          420 non-null    float64 
dtypes: bool(1), category(2), float64(7), int64(5)
memory usage: 120.9 KB


# Cleanup

The columns are named sensibly, without special characters or too long names we've seen in the lectures. Therefore, no changes seem to be needed in that regard.

Another possible area of improvement would be adding new columns to supplement the data. Even though `Losses` and `OPS` (On Base + Slugging) stats can be derived from other columns, they can be easily added to the DataFrame for clarity and simplicity:

In [15]:
# Losses = GamesPlayed - GamesWon
baseball['L'] = baseball.G - baseball.W

In [16]:
baseball['OPS'] = baseball.OBP + baseball.SLG

In [17]:
baseball.L

0        81
1        68
2        69
3        93
4       101
       ... 
1227     80
1228     68
1229     62
1230     79
1231    102
Name: L, Length: 1232, dtype: int64

In [18]:
baseball.OPS

0       0.746
1       0.709
2       0.728
3       0.730
4       0.680
        ...  
1227    0.720
1228    0.715
1229    0.782
1230    0.729
1231    0.681
Name: OPS, Length: 1232, dtype: float64

Now, however, the order of columns is not ideal for viewing:

In [19]:
baseball.columns

Index(['Team', 'League', 'Year', 'RS', 'RA', 'W', 'OBP', 'SLG', 'BA',
       'Playoffs', 'RankSeason', 'RankPlayoffs', 'G', 'OOBP', 'OSLG', 'L',
       'OPS'],
      dtype='object')

To reorder the columns DataFrame, we can reorganize and reassign the columns list as follows:

In [20]:
baseball = baseball[['Team', 'League', 'Year', 'G', 'W', 'L', 'Playoffs', 'RankSeason', 'RankPlayoffs', 'RS', 'RA', 'BA', 'OBP', 'SLG', 'OPS',
         'OOBP', 'OSLG']]

In [21]:
baseball.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Team          1232 non-null   category
 1   League        1232 non-null   category
 2   Year          1232 non-null   int64   
 3   G             1232 non-null   int64   
 4   W             1232 non-null   int64   
 5   L             1232 non-null   int64   
 6   Playoffs      1232 non-null   bool    
 7   RankSeason    244 non-null    float64 
 8   RankPlayoffs  244 non-null    float64 
 9   RS            1232 non-null   int64   
 10  RA            1232 non-null   int64   
 11  BA            1232 non-null   float64 
 12  OBP           1232 non-null   float64 
 13  SLG           1232 non-null   float64 
 14  OPS           1232 non-null   float64 
 15  OOBP          420 non-null    float64 
 16  OSLG          420 non-null    float64 
dtypes: bool(1), category(2), float64(8), int64(6)
memory

With these changes, the dataframe is a bit better organized, with team's overall performance data placed before the more detailed statistics:

In [22]:
baseball

,Team,League,Year,G,W,L,Playoffs,RankSeason,RankPlayoffs,RS,RA,BA,OBP,SLG,OPS,OOBP,OSLG
0,ARI,NL,2012,162,81,81,False,NaN,NaN,734,688,0.259,0.328,0.418,0.746,0.317,0.415
1,ATL,NL,2012,162,94,68,True,4.0,5.0,700,600,0.247,0.320,0.389,0.709,0.306,0.378
2,BAL,AL,2012,162,93,69,True,5.0,4.0,712,705,0.247,0.311,0.417,0.728,0.315,0.403
3,BOS,AL,2012,162,69,93,False,NaN,NaN,734,806,0.260,0.315,0.415,0.730,0.331,0.428
4,CHC,NL,2012,162,61,101,False,NaN,NaN,613,759,0.240,0.302,0.378,0.680,0.335,0.424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,PHI,NL,1962,161,81,80,False,NaN,NaN,705,759,0.260,0.330,0.390,0.720,NaN,NaN
1228,PIT,NL,1962,161,93,68,False,NaN,NaN,706,626,0.268,0.321,0.394,0.715,NaN,NaN
1229,SFG,NL,1962,165,103,62,True,1.0,2.0,878,690,0.278,0.341,0.441,0.782,NaN,NaN
1230,STL,NL,1962,163,84,79,False,NaN,NaN,774,664,0.271,0.335,0.394,0.729,NaN,NaN


In [23]:
# exporting this updated dataframe to a file for use in milestone 3:
baseball.to_csv('baseball_updated.csv')